<a href="https://colab.research.google.com/github/tanatet8/Colab_Script/blob/main/Reasoning_Format_Fix_And_QC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================
# 0) MOUNT GOOGLE DRIVE & GLOBAL SETTINGS
# --------------------------------------------
# ใช้เซลล์นี้เป็นจุดตั้งต้นของโน้ตบุ๊ก:
# - Mount Google Drive เพื่ออ่าน/เขียนไฟล์ได้สะดวก
# - ตั้งชื่อไฟล์ต้นทาง/ปลายทาง
# - ตั้งค่ากติกา format (batch_id, tier mapping)
# - อธิบายภาพรวม "แต่ละบล็อก" ถัดไปไว้เป็นคอมเมนต์
# ============================================

from google.colab import drive
drive.mount('/content/drive')

# --------------------------------------------
# 🔧 ตั้งค่าไฟล์ (แก้เฉพาะบรรทัดนี้ให้เป็นไฟล์ของคุณ)
# --------------------------------------------
SRC = '/content/drive/MyDrive/causal_reasoning_batch_01_FINAL_RENUMBERED_001_500.md'
# ไฟล์ผลลัพธ์ (เวอร์ชันซ่อม format และ tier)
DST_KEEP_NUM   = SRC.replace('.md', '_FIXED_KEEPNUM.md')  # กรณี "คงเลขหัวข้อเดิม"
DST_RENUMBERED = SRC.replace('.md', '_FIXED_RENUMBERED.md')  # กรณี "รีนัมเบอร์ 001..N ใหม่"

# --------------------------------------------
# 📌 กติกา format ที่ต้องเป็นจริง
# --------------------------------------------
# 1) batch_id ที่ถูกต้องสำหรับไฟล์นี้
BATCH_ID_EXPECT = 'causal_reasoning_batch_01'

# 2) mapping reasoning_type -> tier ที่ถูกต้อง
#    (ตัวอย่าง: ทั้ง batch เป็น causal_reasoning จึงต้อง tier: 2 ทั้งหมด)
TIER_MAPPING = {
    'causal_reasoning': 2,
    # ถ้ามี type อื่นในไฟล์ต่อไป เติมได้ เช่น:
    # 'deductive_reasoning': 1,
    # 'probabilistic_reasoning': 2,
}

# --------------------------------------------
# 🧭 ภาพรวม "แต่ละบล็อก" ของโน้ตบุ๊กนี้ (อ่านครั้งเดียวพอ)
# --------------------------------------------
# 1) แบ่งไฟล์เป็นบล็อกด้วยหัวข้อ "## Prompt NNN"
#    - ดึงบล็อกทีละ prompt เพื่อซ่อมเฉพาะจุดแบบไม่พัง format อื่น
# 2) ฟังก์ชันซ่อมบล็อก (fix_block)
#    - บังคับ tier ตาม reasoning_type (อิง TIER_MAPPING)
#    - บังคับ prompt_id ให้ลงท้ายด้วย _NNN ตรงกับหัวข้อ
#    - บังคับ batch_id ให้ตรง BATCH_ID_EXPECT
#    - เติม --- ท้ายบล็อกถ้าขาด
# 3) โหมดซ่อม "คงเลขเดิม" (KEEP NUM)
#    - ใช้กรณีอยากรักษาเลขหัวข้อเดิม (มีเลขซ้ำก็จะคงไว้)
# 4) โหมดซ่อม "รีนัมเบอร์ใหม่" (RENUMBER)
#    - ใช้กรณีเจอเลขซ้ำ/ขาด เช่น "360" ซ้ำ → จะรีเซ็ตหัวข้อให้ 001..N ลำดับใหม่
# 5) วาลิเดชันหลังซ่อม
#    - เช็คซ้ำ/ขาดของหัวข้อ
#    - เช็ค prompt_id suffix ตรงกับหัวข้อไหม
#    - เช็ค batch_id ตรงไหม
#    - เช็ค tier ตรง mapping (เช่น causal ⇒ ต้องเป็น 2) ไหม
#
# 👉 แนวทางใช้งาน:
#   - รันเซลล์นี้ (0) เพื่อ mount และตั้งค่า
#   - รัน "1) แบ่งบล็อก", "2) ฟังก์ชันซ่อม", แล้วเลือก "3) KEEP NUM" หรือ "4) RENUMBER"
#   - ปิดท้ายด้วย "5) วาลิเดชัน" ให้ผ่านทุกข้อก่อนส่งไปทำ Redundancy
#
# ✅ ทิป:
#   - ถ้า batch นี้เป็น causal reasoning ทั้งชุด ให้ตั้ง TIER_MAPPING['causal_reasoning']=2
#   - ถ้ามีหลาย reasoning_type ในไฟล์เดียว ให้ใส่ mapping เพิ่มเพื่อซ่อมทีเดียวจบ
#   - ถ้าไฟล์ยาวและมีเลขหัวข้อซ้ำ (เช่น 360 ซ้ำ) แนะนำใช้โหมด RENUMBER
# --------------------------------------------

print("🎯 Ready!")
print("SRC:", SRC)
print("DST_KEEP_NUM:", DST_KEEP_NUM)
print("DST_RENUMBERED:", DST_RENUMBERED)
print("Expected batch_id:", BATCH_ID_EXPECT)
print("Tier mapping:", TIER_MAPPING)

In [ ]:
# ============================================
# 1) โหลดไฟล์ และ แบ่งออกเป็น block
# --------------------------------------------
# ใช้สำหรับ:
# - อ่านไฟล์ทั้งหมด
# - แบ่งเนื้อหาออกเป็น prompt-block โดยใช้ "## Prompt"
# - เตรียมข้อมูลให้แต่ละ block ซ่อมได้แยกกัน
# ============================================

with open(SRC, 'r', encoding='utf-8') as f:
    text = f.read()

blocks = text.split('## Prompt ')
blocks = [b.strip() for b in blocks if b.strip()]

print(f"พบ {len(blocks)} blocks")
print("ตัวอย่างบล็อกแรก:\n", blocks[0][:500])

In [ ]:
# ============================================
# 2) ฟังก์ชันซ่อม format ของแต่ละ block
# --------------------------------------------
# ใช้สำหรับ:
# - บังคับ batch_id ให้ตรงกับ BATCH_ID_EXPECT
# - บังคับ tier ให้ตรงกับ reasoning_type ใน TIER_MAPPING
# - บังคับ prompt_id ให้ suffix ตรงกับหัวข้อ prompt
# - เติม '---' คั่นท้าย block ถ้าหายไป
# ============================================

import re

def fix_block(block: str) -> str:
    # แก้ batch_id
    block = re.sub(r'(batch_id:\s*)(\S+)', rf'\1{BATCH_ID_EXPECT}', block)

    # แก้ tier ตาม reasoning_type
    m = re.search(r'reasoning_type:\s*(\S+)', block)
    if m:
        rtype = m.group(1)
        if rtype in TIER_MAPPING:
            block = re.sub(r'(tier\s*:\s*)\d+', rf'\1{TIER_MAPPING[rtype]}', block)

    # เติม --- ถ้าขาด
    if not block.strip().endswith('---'):
        block += '\n---\n'

    return block

In [ ]:
# ============================================
# 3) ซ่อมทุก block และบันทึกไฟล์ใหม่
# --------------------------------------------
# ใช้สำหรับ:
# - เรียก fix_block() กับทุก block
# - รวม block กลับเป็นไฟล์เดียว
# - บันทึกเป็นไฟล์ปลายทาง (DST)
# ============================================

fixed_blocks = [fix_block(b) for b in blocks]
fixed_text = ''.join(f"## Prompt {b}" for b in fixed_blocks)

with open(DST, 'w', encoding='utf-8') as f:
    f.write(fixed_text)

print(f"บันทึกไฟล์ซ่อมแล้ว: {DST}")

In [ ]:
# ============================================
# 4) ตรวจสอบความถูกต้องหลังซ่อม
# --------------------------------------------
# ใช้สำหรับ:
# - เช็คว่าทุก block มี batch_id ถูกต้อง
# - tier ถูกตาม mapping
# - ไม่มี block ขาด '---'
# ============================================

import collections

errors = collections.defaultdict(list)

for i, b in enumerate(fixed_blocks, start=1):
    if f"batch_id: {BATCH_ID_EXPECT}" not in b:
        errors['batch_id'].append(i)
    if not b.strip().endswith('---'):
        errors['separator'].append(i)
    m = re.search(r'reasoning_type:\s*(\S+)', b)
    m2 = re.search(r'tier\s*:\s*(\d+)', b)
    if m and m2:
        rtype = m.group(1)
        tier = int(m2.group(1))
        if rtype in TIER_MAPPING and tier != TIER_MAPPING[rtype]:
            errors['tier'].append(i)

print("ผลตรวจ format:", dict(errors))

In [ ]:
# ============================================
# 5) (ภายหลัง) ตรวจ Redundancy
# --------------------------------------------
# ใช้สำหรับ:
# - หลังซ่อม format เสร็จแล้ว
# - นำ reasoning และ prompt จากทุก block มาวัด similarity
# - flag คู่ที่ reasoning similarity >= 0.85 หรือ prompt similarity >= 0.80
# - เตรียม list ให้แก้ซ้ำ
# ============================================
# ⚠️ ยังไม่รันในขั้นนี้ จนกว่าจะ format ผ่าน 100%